In [94]:
import espaloma as esp
import torch
import pinot

In [69]:
g = esp.Graph('c1ccccc1')

In [70]:
from espaloma.data.md import MoleculeVacuumSimulation
simulation = MoleculeVacuumSimulation(
        n_samples=100, n_steps_per_sample=10
)
simulation.run(g)


In [71]:
forcefield = esp.graphs.legacy_force_field.LegacyForceField('smirnoff99Frosst')
forcefield.parametrize(g)


In [124]:
layer = esp.nn.layers.dgl_legacy.gn('SAGEConv')
representation = esp.nn.Sequential(layer, config=[32, 'tanh', 32, 'tanh'])
readout = esp.nn.readout.janossy.JanossyPooling(
    in_features=32, config=[32, 'relu'],
)
net = torch.nn.Sequential(
    representation,
    readout,
    esp.mm.geometry.GeometryInGraph(),
    esp.mm.energy.EnergyInGraph(),
    esp.mm.energy.EnergyInGraph(suffix='_ref'),
)

In [115]:
norm = esp.data.normalize.ESOL100LogNormalNormalize()

Warning (not error because allow_undefined_stereo=True): 
/Users/wangy1/anaconda3/envs/pinot/lib/python3.7/site-packages/dgl/base.py:25: UserWarning: Currently adjacency_matrix() returns a matrix with destination as rows by default.  In 0.5 the result will have source as rows (i.e. transpose=True)
  warnings.warn(msg, warn_type)
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allo

In [127]:
optimizer = torch.optim.Adam(net.parameters(), 1e-3)


In [128]:
for _ in range(1000):
    optimizer.zero_grad()
    def l():
        
        net(g.heterograph)
        norm.unnorm(g.heterograph)
        loss = torch.nn.L1Loss()(
            g.nodes['g'].data['u_ref'],
            g.nodes['g'].data['u'],
        )
        loss.backward()
        print(loss)
        return loss

    optimizer.step(l)

tensor(49.5350, grad_fn=<MeanBackward0>)
tensor(36.0728, grad_fn=<MeanBackward0>)
tensor(27.6019, grad_fn=<MeanBackward0>)
tensor(20.8898, grad_fn=<MeanBackward0>)
tensor(14.7182, grad_fn=<MeanBackward0>)
tensor(11.2120, grad_fn=<MeanBackward0>)
tensor(11.9814, grad_fn=<MeanBackward0>)
tensor(13.5393, grad_fn=<MeanBackward0>)
tensor(13.3076, grad_fn=<MeanBackward0>)
tensor(12.1635, grad_fn=<MeanBackward0>)
tensor(11.4066, grad_fn=<MeanBackward0>)
tensor(11.1565, grad_fn=<MeanBackward0>)
tensor(11.7828, grad_fn=<MeanBackward0>)
tensor(12.0898, grad_fn=<MeanBackward0>)
tensor(11.8931, grad_fn=<MeanBackward0>)
tensor(11.3940, grad_fn=<MeanBackward0>)
tensor(11.1313, grad_fn=<MeanBackward0>)
tensor(11.2965, grad_fn=<MeanBackward0>)
tensor(11.4516, grad_fn=<MeanBackward0>)
tensor(11.4336, grad_fn=<MeanBackward0>)
tensor(11.2472, grad_fn=<MeanBackward0>)
tensor(11.1255, grad_fn=<MeanBackward0>)
tensor(11.3014, grad_fn=<MeanBackward0>)
tensor(11.3906, grad_fn=<MeanBackward0>)
tensor(11.2207, 

KeyboardInterrupt: 

In [78]:
net(g.heterograph)
norm.unnorm(g.heterograph)

Graph(num_nodes={'g': 1, 'n1': 12, 'n2': 24, 'n3': 36, 'n4': 48, 'nonbonded': 6, 'onefour': 48},
      num_edges={('n1', 'n1_neighbors_n1', 'n1'): 24, ('n1', 'n1_as_0_in_n2', 'n2'): 24, ('n2', 'n2_has_0_n1', 'n1'): 24, ('n1', 'n1_as_1_in_n2', 'n2'): 24, ('n2', 'n2_has_1_n1', 'n1'): 24, ('n1', 'n1_as_0_in_n3', 'n3'): 36, ('n3', 'n3_has_0_n1', 'n1'): 36, ('n1', 'n1_as_1_in_n3', 'n3'): 36, ('n3', 'n3_has_1_n1', 'n1'): 36, ('n1', 'n1_as_2_in_n3', 'n3'): 36, ('n3', 'n3_has_2_n1', 'n1'): 36, ('n1', 'n1_as_0_in_n4', 'n4'): 48, ('n4', 'n4_has_0_n1', 'n1'): 48, ('n1', 'n1_as_1_in_n4', 'n4'): 48, ('n4', 'n4_has_1_n1', 'n1'): 48, ('n1', 'n1_as_2_in_n4', 'n4'): 48, ('n4', 'n4_has_2_n1', 'n1'): 48, ('n1', 'n1_as_3_in_n4', 'n4'): 48, ('n4', 'n4_has_3_n1', 'n1'): 48, ('n2', 'n2_as_0_in_n3', 'n3'): 36, ('n3', 'n3_has_0_n2', 'n2'): 36, ('n2', 'n2_as_1_in_n3', 'n3'): 36, ('n3', 'n3_has_1_n2', 'n2'): 36, ('n2', 'n2_as_0_in_n4', 'n4'): 48, ('n4', 'n4_has_0_n2', 'n2'): 48, ('n2', 'n2_as_1_in_n4', 'n4'): 48

In [82]:
g.nodes['n2'].data['k_ref']

tensor([[392459.1875],
        [392459.1875],
        [392459.1875],
        [392459.1875],
        [392459.1875],
        [392459.1875],
        [307105.5938],
        [307105.5938],
        [307105.5938],
        [307105.5938],
        [307105.5938],
        [307105.5938],
        [392459.1875],
        [392459.1875],
        [392459.1875],
        [392459.1875],
        [392459.1875],
        [392459.1875],
        [307105.5938],
        [307105.5938],
        [307105.5938],
        [307105.5938],
        [307105.5938],
        [307105.5938]])